<a href="https://colab.research.google.com/github/takashisuo/signate/blob/master/nishika_hate_s_fasttext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy
!pip install -U ginza ja-ginza
!pip install numpy
!pip intall pandas
!pip install fasttext
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59.1 MB 1.2 MB/s 
     |████████████████████████████████| 2.2 MB 34.9 MB/s 
  Created wheel for SudachiDict-core: filename=SudachiDict_core-20220729-py3-none-any.whl size=71570387 sha256=cb252d2d586ee424c2590c12f0169f5fb973b918b6f639f4fa51d322aed7617b
  Stored in directory: /root/.cache/pip/wheels/ff/47/2b/6dcb905d9086fc6f9b7af821f7145f785d36070b827b7f80b7
Successfully built SudachiDict-core
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: unknown command "intall" - maybe you meant "install"
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 4.1 MB/s 
  Using cached pybind11-2.10.0-py3-none-any.whl (213 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import fasttext as ft
import re
import MeCab
import spacy
import os
import ginza

In [ ]:
root = "/content/drive/MyDrive/Colab Notebooks/nishika/hate/data"
train_path = root + "/train.csv"
train_df = pd.read_csv(train_path)
display(train_df.head())

test_path = root + "/test.csv"
test_df = pd.read_csv(test_path)
display(test_df.head())

submit_path = root + '/sample_submission.csv'
submit_df = pd.read_csv(submit_path)
display(submit_df.head())


,id,source,text,label
0,80074aa43,news4vip,まともに相手されてない人との関係なんて\nそんな大事にするものか？,0
1,6378fea6b,livejupiter,最近はアヘアヘQSマンやない？ ｲｲ!(・∀・)+1-0(・Ａ・)ｲｸﾅｲ!,0
2,c535f5613,livejupiter,日本人として生まれても無能な低学歴って分かったら日本人の権利剥奪して追放すべきやろ\n甘えるな,1
3,e76638295,livejupiter,よくよく思えば川上は配布にしたらとんでもなく有能だよな\nガチャから引いたら圧倒的歓喜レベルやで,0
4,51e4036bf,newsplus,押井は原作レイプの専門家だから\n原作マンガの真意を誤解させることに関してはプロだが\nそれ...,0


,id,source,text
0,001026808,news4vip,上でも言ったけどオタクレベルの知識求めてる訳じゃない\nただ囲碁やります！って人が誰1人プロ...
1,00465ac96,livejupiter,たとえば、黒人なんかは、生物学的欠陥はないのに、文化的要因で、悪循環に陥り、実力をつけられず...
2,004674725,livejupiter,そうなんやろなあ色々と勿体ない感じしたわ\n終わり方と黒幕キャラは好きやったで\n\nちなワ...
3,00474460f,news4vip,法的というか自治体ごとにバラバラの条例で定めてるだけだからな\n普通の淫行条例だと「青少年に...
4,004a7525c,newsplus,別のジャーナリストの感想として言われてるので客観的な事実とは言えないけど、\n現地は不測の事...


,id,label
0,001026808,0
1,00465ac96,0
2,004674725,0
3,00474460f,0
4,004a7525c,0


In [ ]:
train_df["label"].value_counts()
# hateの水増しが必要な可能性あり★

0    4950
1     306
Name: label, dtype: int64

In [ ]:
# 改行は除去しておく。
def remove_linefeed(words):
  return re.sub(r"[\t\r\n]", " ", words)


# 改行削除
train_df["text"] = train_df["text"].map(remove_linefeed)
test_df["text"] = test_df["text"].map(remove_linefeed)

display(train_df.head())

,id,source,text,label
0,80074aa43,news4vip,まともに相手されてない人との関係なんて そんな大事にするものか？,0
1,6378fea6b,livejupiter,最近はアヘアヘQSマンやない？ ｲｲ!(・∀・)+1-0(・Ａ・)ｲｸﾅｲ!,0
2,c535f5613,livejupiter,日本人として生まれても無能な低学歴って分かったら日本人の権利剥奪して追放すべきやろ 甘えるな,1
3,e76638295,livejupiter,よくよく思えば川上は配布にしたらとんでもなく有能だよな ガチャから引いたら圧倒的歓喜レベルやで,0
4,51e4036bf,newsplus,押井は原作レイプの専門家だから 原作マンガの真意を誤解させることに関してはプロだが それ以外...,0


In [ ]:
class FastTextProxy:

  def __init__(self, train_df, test_df, submit_df):
    self.train_df = train_df.copy()
    self.test_df = test_df.copy()
    self.submit_df = submit_df.copy()
    self.train_data = None
    self.model = None

  def train(self):
    if not os.path.exists(root + '/fasttext/train_df.txt'):
      self.transform_train(self.train_df)
    if not os.path.exists(root + '/fasttext/model.bin'):
      self.model = ft.train_supervised(root + '/fasttext/train_df.txt', label_prefix='__label__', epoch=300, loss="hs")
      self.model.save_model(root + '/fasttext/model.bin')

  def load(self):
    self.model = ft.load_model(root + '/fasttext/model.bin')


  def pred_labels(self):
    org_df = self.submit_df.copy()

    self.test_df = self.transform_test(self.test_df)
    display(self.test_df.head(3))
    #self.test_df['label'] = None
    self.submit_df['label'] = self.test_df['text'].map(self.pred)
    display(self.submit_df.head(2))
    self.submit_df['label'] = self.submit_df['label'].map(self.__remove_label)
    self.submit_df.to_csv(os.path.join(root, 'fasttext','submit_fasttext.csv'), index=None) 


  def transform_train(self, df):
    df['label'] = df['label'].map(self.__modify)

    # わかち書き(原型)
    nlp = spacy.load("ja_ginza")
    ginza.set_split_mode(nlp, "B")
    df["text"] = df["text"].apply(lambda x: self.wakati(nlp, x))
    df['text'] = df['source'] + ' ' + df['text']

    df.drop(['id', 'source'], axis=1, inplace=True)
    df = df[['label', 'text']]
    display(df.head())
    df.to_csv(root + '/fasttext/train_df.txt', index=None, header=None, sep='\t')

  def wakati(self, nlp, txt):
      doc = nlp(txt)
      words = []
      for sent in doc.sents:
          for token in sent:
            words.append(token.text)
      return " ".join(words)


  def transform_test(self, df):
    #df.drop(['id', 'source'], axis=1, inplace=True)
    # わかち書き(原型)
    nlp = spacy.load("ja_ginza")
    ginza.set_split_mode(nlp, "B")
    df["text"] = df["text"].apply(lambda x: self.wakati(nlp, x))
    df['text'] = df['source'] + ' ' + df['text']

    df = df[['text']]
    display(df.head())
    df.to_csv(root + '/fasttext/test_df.txt', index=None, header=None, sep='\t')
    return df


  def __modify(self, x):
    return '__label__' + str(x)

  def __remove_label(self, x):
    return x.replace('__label__', '')

  def pred(self, x):
    res = self.model.predict(x)
    return res[0][0]

In [ ]:
# 検証
def wakati(nlp, txt):
    doc = nlp(txt)
    words = []
    for sent in doc.sents:
        for token in sent:
          print(token.text)
          words.append(token.text)
    return " ".join(words)

nlp = spacy.load("ja_ginza")
wakati(nlp, "これはテストですよね。")

これ
は
テスト
です
よ
ね
。


'これ は テスト です よ ね 。'

In [ ]:
ftp = FastTextProxy(train_df, test_df, submit_df)
ftp.train()

,label,text
0,__label__0,news4vip まとも に 相手 さ れ て ない 人 と の 関係 なんて そんな 大事...
1,__label__0,livejupiter 最近 は アヘアヘ QS マン や ない ？ ｲｲ ! (・∀・...
2,__label__1,livejupiter 日本人 と し て 生まれ て も 無能 な 低学歴 って 分かっ ...
3,__label__0,livejupiter よくよく 思え ば 川上 は 配布 に し たら とんでも なく 有...
4,__label__0,newsplus 押井 は 原作 レイプ の 専門家 だ から 原作 マンガ の 真意 を ...


In [ ]:
ftp.load()
#ftp.test()
# train_test_splitをおこない検証をおこなう

In [ ]:
test_df.head()

,id,source,text
0,001026808,news4vip,上でも言ったけどオタクレベルの知識求めてる訳じゃない ただ囲碁やります！って人が誰1人プロ棋...
1,00465ac96,livejupiter,たとえば、黒人なんかは、生物学的欠陥はないのに、文化的要因で、悪循環に陥り、実力をつけられず...
2,004674725,livejupiter,そうなんやろなあ色々と勿体ない感じしたわ 終わり方と黒幕キャラは好きやったで ちなワイはダ...
3,00474460f,news4vip,法的というか自治体ごとにバラバラの条例で定めてるだけだからな 普通の淫行条例だと「青少年に淫...
4,004a7525c,newsplus,別のジャーナリストの感想として言われてるので客観的な事実とは言えないけど、 現地は不測の事態...


In [ ]:
ftp.pred_labels()

,text
0,news4vip 上 で も 言っ た けど オタク レベル の 知識 求め てる 訳 じゃ...
1,livejupiter たとえば 、 黒人 なんか は 、 生物学 的 欠陥 は ない の ...
2,livejupiter そう なん やろ なあ 色々 と 勿体 ない 感じ し た わ 終わ...
3,news4vip 法的 と いう か 自治体 ごと に バラバラ の 条例 で 定め てる ...
4,newsplus 別 の ジャーナリスト の 感想 と し て 言わ れ てる の で 客観...


,text
0,news4vip 上 で も 言っ た けど オタク レベル の 知識 求め てる 訳 じゃ...
1,livejupiter たとえば 、 黒人 なんか は 、 生物学 的 欠陥 は ない の ...
2,livejupiter そう なん やろ なあ 色々 と 勿体 ない 感じ し た わ 終わ...


,id,label
0,001026808,__label__0
1,00465ac96,__label__0


In [ ]:
res_df = pd.read_csv(root + '/fasttext/submit_fasttext.csv')
res_df["label"].value_counts()

0    3084
1     139
Name: label, dtype: int64

In [ ]:
model = ftp.model
model.predict("おっ、やっぱり反対になったか これは国が動けという合図だな 支那,朝鮮,在日を国家主導で亡ぼさないと日本は進化できないぞ")

(('__label__0',), array([1.00001001]))